In [1]:
import bs4
import requests
import pandas as pd

In [2]:
def collect_name(init_page,last_page):

    df_brand=pd.DataFrame()
    df_product=pd.DataFrame()
    df_link=pd.DataFrame()
    
    for num in range(init_page,last_page+1):
        link=requests.get('https://www.skincarisma.org/search?category=Face+Skincare&page='+str(num))
        html_page = bs4.BeautifulSoup(link.text, 'lxml')
        find_data=html_page.find('ul',{'class':'list-unstyled mt-2 mb-0'})

    #collect brand
        find_name=find_data.find_all('b',{'itemprop':'brand'})
        for i in find_name:
            name=i.text.strip()
            df_brand = df_brand.append({'brand': name}, ignore_index=True)
            print('page =',num,'brand =',name)
            
    #collect product
        find_product=find_data.find_all('div',{'itemprop':'name'})
        for i in find_product:
            name=i.select('p')
            name=name[0].text
            df_product = df_product.append({'product': name}, ignore_index=True)

    #collect link
        find_link=find_data.find_all('div',{'class':'d-inline-block image pl-3'})
        for i in find_link:
            name=i.find('a')['href']
            df_link = df_link.append({'link': name}, ignore_index=True)

    df=pd.concat([df_brand,df_product,df_link],axis=1)
    return df

In [3]:
def collect_ing(df):
    df_ingredient=pd.DataFrame()
    df_category=pd.DataFrame()
    ing_tag_per=dict()
    ing_cir_per=dict()
    ing_ewg_per=dict()
    check=[]
    index_error=[]

    for ind,url in enumerate(df['link']):
        print(len(df),':',ind,':',url)
        link=requests.get('https://www.skincarisma.org/'+url+'/ingredient_list')
        html_page = bs4.BeautifulSoup(link.text, 'lxml')
        find_data=html_page.find_all('td',class_="align-middle")
        if find_data:          
            find_cate=html_page.find_all('div',class_='col-12 mb-2')
       #collect type of product 
            cate=[ind_cate.text for ind_cate in find_cate[0].select('span')]          
        
            ing_tag_temp=dict()
            for i in find_data:
                find_cir=i.find_all('div',class_='score-label cir')
                find_ewg=i.find_all('div',class_='score-label')
        #find cir of each ingredient
                if find_cir:
                    cir_check=find_cir[0].select('span')
                    if cir_check:
                        cir=cir_check[0].text
                    else:
                        cir=''
        #find ewg of each ingredient
                if '<div class="score-label">' in str(i) or '<div class="score-label second">' in str(i):
                    if '<div class="score-label second">' not in str(i):
                        ewg_check=find_ewg[0].select('span')
                        if ewg_check:
                            ewg=ewg_check[0].text
                        else:
                            ewg=''
            #there are 2 ewg, collect the highest ewg value
                    else:
                        ewg_check=find_ewg[0].select('span')
                        if ewg_check:
                            ewg_max=list([int(ewg_check[0].text),int(ewg_check[1].text)])
                            ewg_max=str(max(ewg_max))
                            ewg=ewg_max
                        else:
                            ewg=''
        #find ingredient name
                if '<td class="align-middle">' in str(i) and '<br/>' in str(i) and '</span>' not in str(i):
                    for j in i:
                        if '<' not in str(j) and len(j.strip())>0:
                            name=j.strip()
                    tag=i.select('small')[0].text.strip()
                    tag=tag[1:-1]
                    tag=set(tag.split(','))
                    ing_tag_temp.update({name:tag})
        #collect tag of each ingredient
                    ing_tag_per.update({name:tag})
        #collect cir of each ingredient (if duplicated, collect more danger level (C>B>A))
                    if name in ing_cir_per and ewg!=ing_cir_per.get(name):
                        if ewg=='C' or ing_ewg_per.get(name)=='C':
                            ing_cir_per.update({name:'C'})
                        else:
                            if ewg=='B' or ing_ewg_per.get(name)=='B':
                                ing_cir_per.update({name:'B'})
                            elif ewg=='A' or ing_ewg_per.get(name)=='A':
                                ing_cir_per.update({name:'A'})
                    else:
                        ing_cir_per.update({name:cir})
        #collect ewg of each ingredient (if duplicated, collect the highest ewg value)
                    if name in ing_ewg_per and ewg!='' and ing_ewg_per.get(name)!='':
                        if int(ewg)>int(ing_ewg_per.get(name)):
                            ing_ewg_per.update({name:ewg})
                        else:
                            if ewg!='':               
                                ing_ewg_per.update({name:ewg})
                    elif name not in ing_ewg_per:
                        ing_ewg_per.update({name:ewg})
        #collect list of ingredient for each items
            ing=set(ing_tag_temp.keys())
            df_ingredient = df_ingredient.append({'ingredient': ing}, ignore_index=True)
            df_category = df_category.append({'category': cate}, ignore_index=True)
            check.append(0)
        
        else:
            df_ingredient = df_ingredient.append({'ingredient': None}, ignore_index=True)
            df_category = df_category.append({'category': None}, ignore_index=True)
            check.append(1)
            index_error.append(ind)
                                       
    return df_ingredient,df_category,ing_tag_per,ing_cir_per,ing_ewg_per,check,index_error

In [4]:
def remove_item(df,ind):
    df=df.drop(ind)
    return df

In [5]:
def create_table(init_page,last_page):
    df_product=collect_name(init_page,last_page)
    table=collect_ing(df_product)
    df_ingredient=table[0]
    df_category=table[1]
    
    if sum(table[5])>0:
        for ind_err in table[6]:
            df_product=remove_item(df_product,ind_err)
            df_ingredient=remove_item(table[0],ind_err)
            df_category=remove_item(table[1],ind_err)

    df=pd.concat([df_product,df_ingredient,df_category],axis=1)
#save product detail to csv file    
    df.to_csv('data\product\product_'+str(init_page)+'-'+str(last_page)+'.csv', index = None, header=False)
    
    ing=[]
    tag=[]
    cir=[]
    ewg=[]
    for i in table[2]:
        ing.append(i)
        tag.append(table[2].get(i))
        cir.append(table[3].get(i))
        ewg.append(table[4].get(i))
    ing_table={'ingredient':ing,'tag':tag,'cir':cir,'ewg':ewg}
    ing_table=pd.DataFrame(ing_table)
    
    ing_table.to_csv('data\ingredient\ingredient_'+str(init_page)+'-'+str(last_page)+'.csv', index = None, header=False)
    return df,ing_table

In [6]:
#there are 1200 pages
table=create_table(1,10)
table[0]

page = 1 brand = Dermalogica
page = 1 brand = Neutrogena
page = 1 brand = Aveeno
page = 1 brand = L'Oreal Paris
page = 1 brand = Hada Labo
page = 1 brand = MISSHA
page = 1 brand = LADYKIN
page = 1 brand = Clinique
page = 1 brand = Olay
page = 1 brand = Estee Lauder
page = 1 brand = SKINFOOD
page = 1 brand = SKINFOOD
page = 1 brand = Purito
page = 1 brand = Stridex
page = 1 brand = TIA'M
page = 1 brand = Olay
page = 1 brand = Purito
page = 1 brand = Estee Lauder
page = 1 brand = Estee Lauder
page = 1 brand = Aveeno
page = 1 brand = ETUDE HOUSE
page = 1 brand = Olay
page = 1 brand = Shiseido
page = 1 brand = The Body Shop
page = 1 brand = Estee Lauder
page = 2 brand = Laneige
page = 2 brand = IsNtree
page = 2 brand = Mizon
page = 2 brand = Biore
page = 2 brand = Clinique
page = 2 brand = Laneige
page = 2 brand = Clinique
page = 2 brand = Clinique
page = 2 brand = Hada Labo
page = 2 brand = Eucerin
page = 2 brand = ETUDE HOUSE
page = 2 brand = Eucerin
page = 2 brand = Dermalogica
page = 2

250 : 21 : /products/olay/oil-minimizing-toner
250 : 22 : /products/shiseido/the-skincare-purifying-mask
250 : 23 : /products/the-body-shop/for-men-maca-root-energetic-face-protector-spf-15
250 : 24 : /products/estee-lauder/verite-lightlotion-cleanser
250 : 25 : /products/laneige/essential-power-skin-refiner_moisture
250 : 26 : /products/isntree/hyaluronic-acid-toner
250 : 27 : /products/mizon/no-1-moist-kings-berry-cream
250 : 28 : /products/biore/skin-care-facial-foam-rich-moisture
250 : 29 : /products/clinique/pep-start-2-in-1-exfoliating-cleanser
250 : 30 : /products/laneige/essential-balancing-emulsion_moisture
250 : 31 : /products/clinique/facial-soap-mild
250 : 32 : /products/clinique/clarifying-lotion-4
250 : 33 : /products/hada-labo/super-hyaluronic-acid-hydrating-lotion-light
250 : 34 : /products/eucerin/redness-relief-daily-perfecting-lotion-spf-15
250 : 35 : /products/etude-house/real-art-cleansing-oil-perfect
250 : 36 : /products/eucerin/sensitive-skin-redness-relief-daily

250 : 147 : /products/cosrx/one-step-green-hero-calming-pad
250 : 148 : /products/dr-morita/hyaluronic-acid-long-lasting-facial-mask
250 : 149 : /products/kiehl-s/ultra-facial-oil-free-cleanser
250 : 150 : /products/laneige/white-dew-tone-up-cream
250 : 151 : /products/the-ordinary/amino-acids-b5
250 : 152 : /products/ren/ready-steady-glow-daily-aha-tonic
250 : 153 : /products/freeman/feeling-beautiful-coconut-glow-leave-on-bronzing-mask
250 : 154 : /products/paula-s-choice/10-azelaic-acid-booster
250 : 155 : /products/cezanne/skin-conditioner-high-moist
250 : 156 : /products/ilia/true-skin-serum-foundation
250 : 157 : /products/paula-s-choice/c15-super-booster
250 : 158 : /products/somethinc/level-1-retinol
250 : 159 : /products/clinique/smart-treatment-oil
250 : 160 : /products/estee-lauder/re-nutriv-ultimate-lift-age-correcting-serum
250 : 161 : /products/leaders/tea-tree-relaxing-mask
250 : 162 : /products/lush/herbalism
250 : 163 : /products/thayers/alcohol-free-cucumber-witch-haz

,brand,product,link,ingredient,category
0,Dermalogica,After Sun Repair,/products/dermalogica/after-sun-repair,"{Cucumis Sativus (Cucumber) Fruit Extract, Hyd...","[Face Skincare, Facial Suncare, Aftersun]"
1,Neutrogena,Foot Cream Norwegian Formula,/products/neutrogena/foot-cream-norwegian-formula,"{Keratin Amino Acids, Panthenol, Cyclomethicon...","[Face Skincare, Moisturizers]"
2,Aveeno,"Soothing Relief Moisture Cream, Fragrance Free",/products/aveeno/soothing-relief-moisture-crea...,"{Avena Sativa (Oat) Kernel Flour, Benzalkonium...","[Face Skincare, Moisturizers]"
3,L'Oreal Paris,"Sublime Glow Daily Moisturizer, for Fair Skin ...",/products/l-oreal-paris/sublime-glow-daily-moi...,"{Mangifera Indica (Mango) Seed Oil, PEG-100 St...","[Face Skincare, Moisturizers]"
4,Hada Labo,Gokujyun Lotion Refill Pouch,/products/hada-labo/gokujyun-lotion-refill-pouch,"{Succinic Acid, Glycerin, Butylene Glycol, Sod...","[Face Skincare, Toners]"
...,...,...,...,...,...
245,Estee Lauder,Re Nutriv Ultimate Lift Age Correcting Creme,/products/estee-lauder/re-nutriv-ultimate-lift...,"{Cucumis Sativus (Cucumber) Fruit Extract, Di-...","[Face Skincare, Moisturizers]"
246,Acne.org,Treatment,/products/acne-org/treatment,"{Butylene Glycol, Benzoyl Peroxide, Sodium Hyd...","[Face Skincare, Treatments, Acne & Blemish Tre..."
247,Neutrogena,Deep Moisture Night Cream,/products/neutrogena/deep-moisture-night-cream,"{Cetearyl Alcohol, Caprylic/Capric Triglycerid...","[Face Skincare, Moisturizers]"
248,Banila Co.,Clean It Zero Radiance,/products/banila-co/clean-it-zero-radiance,"{PEG-10 Isostearate, Collagen, Glycerin, Butyl...","[Face Skincare, Cleansers, Cleansing Oil & Balms]"


In [7]:
table[1]

,ingredient,tag,cir,ewg
0,Algae Extract,"{Skin Conditioning, Fragrance, Humectant, Emol...",,1
1,Zanthoxylum Alatum Fruit Extract,"{Masking, Skin Conditioning}",,1
2,Glycolipids,{Skin Conditioning},,1
3,Glycereth-7 Triacetate,{Emollient},,1
4,Dimethiconol,"{Antifoaming Agent, Emollient, Skin Conditioni...",A,1
...,...,...,...,...
1602,Moringa Pterygosperma Seed Extract,{Skin Conditioning},,1
1603,Collagen,"{Hair Conditioning, Skin Conditioning}",,1
1604,Vitis Vinifera (Grape) Root Extract,{},A,1
1605,Glyceryl Oleate,"{Emulsifying, Emollient, Surfactant, Fragrance...",A,1


In [8]:
#table=create_table(11,20)
#table[0]

In [9]:
#table[1]

In [10]:
#table=create_table(21,30)
#table[0]

In [11]:
#table[1]

In [12]:
#table=create_table(31,40)
#table[0]

In [13]:
#table[1]

In [14]:
#table=create_table(41,50)
#table[0]

In [15]:
#table[1]

In [16]:
#table=create_table(51,60)
#table[0]

In [17]:
#table[1]

In [18]:
#table=create_table(61,70)
#table[0]

In [19]:
#table[1]

In [20]:
#table=create_table(71,80)
#table[0]

In [21]:
#table[1]

In [22]:
#table=create_table(81,90)
#table[0]

In [23]:
#table[1]

In [24]:
#table=create_table(91,100)
#table[0]

In [25]:
#table[1]